In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-9713d571-c54d-5f22-cf70-5859a3c75be8)


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-7b1",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:


for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)



In [ ]:


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )



In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 7864320 || all params: 7076880384 || trainable%: 0.11112693126452029


In [ ]:
import transformers
from datasets import load_dataset

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1y5syUvsPOmWQNLZwRnkhIUh9PuPbOKa-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1y5syUvsPOmWQNLZwRnkhIUh9PuPbOKa-" -O data.zip && rm -rf /tmp/cookies.txt

--2023-12-13 19:43:10--  https://docs.google.com/uc?export=download&confirm=t&id=1y5syUvsPOmWQNLZwRnkhIUh9PuPbOKa-
Resolving docs.google.com (docs.google.com)... 142.250.4.113, 142.250.4.101, 142.250.4.138, ...
Connecting to docs.google.com (docs.google.com)|142.250.4.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/g9q9nnemn598agd1r3ch8nr46iplplhq/1702496550000/07653173915231569853/*/1y5syUvsPOmWQNLZwRnkhIUh9PuPbOKa-?e=download&uuid=9e7faf91-09ff-4f33-8732-499a48b7c75d [following]
--2023-12-13 19:43:10--  https://doc-00-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/g9q9nnemn598agd1r3ch8nr46iplplhq/1702496550000/07653173915231569853/*/1y5syUvsPOmWQNLZwRnkhIUh9PuPbOKa-?e=download&uuid=9e7faf91-09ff-4f33-8732-499a48b7c75d
Resolving doc-00-3g-docs.googleusercontent.com (doc-00-3g-docs.googleusercontent.com)... 172.217.194.132, 240

In [ ]:
!unzip data.zip


Archive:  data.zip
replace final_captioned_dataset.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: final_captioned_dataset.xlsx  


In [ ]:
df = pd.read_excel('final_captioned_dataset.xlsx')

In [ ]:
df.columns

Index(['Unnamed: 0', 'id', 'date', 'likes', 'content', 'username', 'media',
       'inferred company', 'image_path', 'image_caption'],
      dtype='object')

In [ ]:
df = df.drop('Unnamed: 0',axis = 1)

In [ ]:
df

,id,date,likes,content,username,media,inferred company,image_path,image_caption
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,1.jpg,a close up of a sandwich and a drink on a table
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,2.jpg,a woman with dreadlocks is speaking into a mic...
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,3.jpg,there is a man in a green hat and a green helm...
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,4.jpg,people are on the beach with umbrellas and cha...
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,5.jpg,arafed man and woman standing in front of a cr...
...,...,...,...,...,...,...,...,...,...
299995,299996,2019-09-07 16:18:10,0,Barcelona Star Expected To Return Against Vale...,IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,299996.jpg,barcelona's players celebrate after scoring a ...
299996,299997,2018-02-23 11:24:36,46,Kjeld Nuis of #NED is golden again... This tim...,CBCOlympics,[Video(thumbnailUrl='https://pbs.twimg.com/med...,cbc,299997.jpg,arafed man in orange shirt playing tennis on c...
299997,299998,2020-11-11 20:18:15,261,Grateful 🙌🏾 to have <mention> officially join ...,TeamCoachBuzz,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,299998.jpg,a cartoon of a basketball player with a signed...
299998,299999,2019-10-29 10:44:00,119,The US Air Force's mysterious space plane just...,Independent,[Video(thumbnailUrl='https://pbs.twimg.com/med...,independent,299999.jpg,araflane on the tarmac at night with a truck a...


In [ ]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

In [ ]:
result_df = df.drop(df.index[150000:200000])

In [ ]:
result_df.reset_index(inplace = True)

In [ ]:
train = result_df.iloc[:220000]

In [ ]:
val = result_df.iloc[220000:]

In [ ]:
train.drop(['index','id','image_path'],axis = 1,inplace =True)

<ipython-input-22-12b6e0c8568d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['index','id','image_path'],axis = 1,inplace =True)


In [ ]:
val.drop(['index','id','image_path'],axis = 1,inplace =True)

<ipython-input-23-283283595c6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.drop(['index','id','image_path'],axis = 1,inplace =True)


In [ ]:
train.drop('media',axis = 1, inplace = True)
val.drop('media',axis = 1, inplace = True)

<ipython-input-24-b06ee69b0cff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('media',axis = 1, inplace = True)
<ipython-input-24-b06ee69b0cff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.drop('media',axis = 1, inplace = True)


In [ ]:
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(val)

In [ ]:

data = DatasetDict()

data['train'] = tds
data['validation'] = vds

In [ ]:
train.image_caption.isnull().sum()

3213

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['date', 'likes', 'content', 'username', 'inferred company', 'image_caption'],
        num_rows: 220000
    })
    validation: Dataset({
        features: ['date', 'likes', 'content', 'username', 'inferred company', 'image_caption'],
        num_rows: 30000
    })
})

In [ ]:
def clean(example):
  if example['image_caption'] is None:
    example['image_caption'] = ""
  return example

In [ ]:
data['train'] = data['train'].map(clean)

Map:   0%|          | 0/220000 [00:00<?, ? examples/s]

In [ ]:
data['validation'] = data['validation'].map(clean)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
def merge_columns(example):
    example["prediction"] = example["inferred company"] +" with username " + example['username'] +  " posted a tweet at " + str(example['date']) + " with an image that is " + example['image_caption'] + " with number of likes " + str(example['likes'])+ " that reads ->: " + str(example["content"])
    return example

data['train'] = data['train'].map(merge_columns)
data['validation'] = data['validation'].map(merge_columns)

data['train']["prediction"][:5]

Map:   0%|          | 0/220000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

['tim hortons with username TimHortonsPH posted a tweet at 2020-12-12 00:47:00 with an image that is a close up of a sandwich and a drink on a table with number of likes 1 that reads ->: Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>',
 'independent with username IndyMusic posted a tweet at 2018-06-30 10:04:20 with an image that is a woman with dreadlocks is speaking into a microphone with number of likes 2750 that reads ->: Watch rapper <mention> freestyle for over an HOUR <hyperlink> <hyperlink>',
 'cbc with username CBCCanada posted a tweet at 2020-09-29 19:47:28 with an image that is there is a man in a green hat and a green helmet working on a pile of pipes with number of likes 57 that reads ->: Canadian Armenian community demands ban on military drone tech exports to Turkey <hyperlink> <hyperlink>',
 "williams with username MKWilliamsRome posted a

In [ ]:
data['train'][0]

{'date': '2020-12-12 00:47:00',
 'likes': 1,
 'content': 'Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>',
 'username': 'TimHortonsPH',
 'inferred company': 'tim hortons',
 'image_caption': 'a close up of a sandwich and a drink on a table',
 'prediction': 'tim hortons with username TimHortonsPH posted a tweet at 2020-12-12 00:47:00 with an image that is a close up of a sandwich and a drink on a table with number of likes 1 that reads ->: Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>'}

In [ ]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/220000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['date', 'likes', 'content', 'username', 'inferred company', 'image_caption', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 220000
    })
    validation: Dataset({
        features: ['date', 'likes', 'content', 'username', 'inferred company', 'image_caption', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 30000
    })
})

In [ ]:
%time
trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=400,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,4.533900
2,4.326200
3,4.336300
4,4.391600
5,4.350600
6,4.406200
7,4.599800
8,4.531400
9,4.461700
10,4.496200


In [ ]:
model.push_to_hub("MohdRehan001/bloom-7b1-lora-tagger-for-tweet-generation",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)



/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:821: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MohdRehan001/bloom-7b1-lora-tagger-for-tweet-generation/commit/98c238c1e504fb9594b4692a2305be8fb2df4224', commit_message='basic training', commit_description='', oid='98c238c1e504fb9594b4692a2305be8fb2df4224', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "MohdRehan001/bloom-7b1-lora-tagger-for-tweet-generation"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)



In [ ]:
data['train'][1002]['prediction'].split('->:')

['cnn with username CNN posted a tweet at 2018-07-23 06:30:06 with an image that is a close up of a magazine cover with a man in a suit and tie with number of likes 981 that reads ',
 ' For the second month in a row, Time magazine has put a provocative photo illustration of President Trump on its cover -- this time, morphing him into Russian President Vladimir <hyperlink> <hyperlink>']

In [ ]:
txt = data['train'][1002]['prediction'].split('->:')[0]

In [ ]:
txt

'cnn with username CNN posted a tweet at 2018-07-23 06:30:06 with an image that is a close up of a magazine cover with a man in a suit and tie with number of likes 981 that reads '

In [ ]:
batch = tokenizer(txt + " ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1665: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkp



 cnn with username CNN posted a tweet at 2018-07-23 06:30:06 with an image that is a close up of a magazine cover with a man in a suit and tie with number of likes 981 that reads  ->:  The new #CNNPolitics cover features a photo of President Trump and a quote from him: "When you have a great leader, you have a great country." <hyperlink> <hyperlink> <hyperlink> <hyperlink>
